# Trabalho Prático

## Linear Regression Project

## Imports
**Import pandas, numpy, matplotlib,and seaborn. Then set %matplotlib inline 
(You'll import sklearn as you need it.)**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, plot_confusion_matrix
import re
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier




# Read Dataset

In [ ]:
train = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')

# Data Preparation

In [ ]:
print(len(test))
print(len(train))

### Remove City Column

In [ ]:
citiesValues = train.groupby(['city_name'])['city_name'].count()
print(citiesValues)

In [ ]:
train = train.drop(['city_name'],axis=1)
test = test.drop(['city_name'],axis=1)


### Magnitude of delay treatment

In [ ]:
magnitude = train.groupby(['magnitude_of_delay'])['magnitude_of_delay'].count()
test.groupby(['magnitude_of_delay'])['magnitude_of_delay'].count()
print(magnitude)

In [ ]:
train = train.drop(['magnitude_of_delay'],axis=1)
test = test.drop(['magnitude_of_delay'],axis=1)

### Delay in Seconds

In [ ]:
# def absolute_maximum_scale(series):
#     return series / series.abs().max()


# train['delay_in_seconds'] = absolute_maximum_scale(train['delay_in_seconds'])
# test['delay_in_seconds'] = absolute_maximum_scale(test['delay_in_seconds'])


### Avg_Precipitaion

In [ ]:
precipitation = train.groupby(['avg_precipitation'])['avg_precipitation'].count()
print(precipitation)

train = train.drop(['avg_precipitation'],axis=1)
test = test.drop(['avg_precipitation'],axis=1)

### Label Enconding

In [ ]:
luminosity_values = train.groupby(['luminosity'])['luminosity'].count()
print(luminosity_values)

rain_values = train.groupby(['avg_rain'])['avg_rain'].count()
print(rain_values)

label_encoder = preprocessing.LabelEncoder()
train['luminosity'] = label_encoder.fit_transform(train['luminosity'])
train['avg_rain'] = label_encoder.fit_transform(train['avg_rain'])
# train['delay_in_seconds'] = label_encoder.fit_transform(train['delay_in_seconds'])



test['avg_rain'] = label_encoder.fit_transform(test['avg_rain'])
test['luminosity'] = label_encoder.fit_transform(test['luminosity'])
# test['delay_in_seconds'] = label_encoder.fit_transform(test['delay_in_seconds'])


### Affected Roads

In [ ]:
roads = train.groupby(['affected_roads'])['affected_roads'].count()

def hotLabelEncoding(dataset):
    dict = {}
    dict['total_estradas'] = []
        
    columns = dataset['affected_roads']

    for column in columns:
        if (column != '' and isinstance(column, str) ):
            roadList = column.split(",")
            for road in roadList:
                if (road != '' and road != 'nan' and road != 'IC5 - N206' and road != 'N101 - N310' and road != 'N207-4'):
                    dict[road] = []
                    
    
    for line in dataset['affected_roads']:
        if (line != '' and isinstance(line, str) ):
            count = 0
            total_count = 0
            for road in dict.keys() :
                if(road != 'total_estradas'):
                    count = line.count(road)
                    total_count = total_count + count
                    dict[road].append(count)
            dict['total_estradas'].append(total_count)
        else:
            for road in dict.keys():
                dict[road].append(0)


    for road in sorted(dict.keys(), key=lambda x:x.lower()):
        if road == 'total_estradas':
             dataset[road] = dict[road]
    

hotLabelEncoding(dataset=train)
hotLabelEncoding(dataset=test)


train = train.drop(['affected_roads'],axis=1)
test = test.drop(['affected_roads'],axis=1)







### Data and hour treatment

In [ ]:
# separate collumn Date 
def separate_year(time):
    res = re.search(r'(\d+)-(\d+)-(\d+) (\d+):(\d+)', time)
    if res:
        return int(res.group(1))

def separate_month(time):
    res = re.search(r'(\d+)-(\d+)-(\d+) (\d+):(\d+)', time)
    if res:
        return int(res.group(2))
    
def separate_day(time):
    res = re.search(r'(\d+)-(\d+)-(\d+) (\d+):(\d+)', time)
    if res:
        return int(res.group(3))
    
def separate_hour(time):
    res = re.search(r'(\d+)-(\d+)-(\d+) (\d+):(\d+)', time)
    if res:
        return int(res.group(4))
        
def separate_minutes(time):
    res = re.search(r'(\d+)-(\d+)-(\d+) (\d+):(\d+)', time)
    if res:
        return int(res.group(5))
    
train['Year'] = train['record_date'].apply(separate_year)
train['Month'] = train['record_date'].apply(separate_month)
train['Day'] = train['record_date'].apply(separate_day)
train['Hour'] = train['record_date'].apply(separate_hour)
train['Minutes'] = train['record_date'].apply(separate_minutes)
minutes_values = train.groupby(['Minutes'])['Minutes'].count()
print(minutes_values)

train = train.drop(['record_date'],axis=1)


test['Year'] = test['record_date'].apply(separate_year)
test['Month'] = test['record_date'].apply(separate_month)
test['Day'] = test['record_date'].apply(separate_day)
test['Hour'] = test['record_date'].apply(separate_hour)
test['Minutes'] = test['record_date'].apply(separate_minutes)
test = test.drop(['record_date'],axis=1)

test = test.drop(['Minutes'],axis=1)
train = train.drop(['Minutes'],axis=1)

test = test.drop(['Year'],axis=1)
train = train.drop(['Year'],axis=1)



# Data Vizualization

In [ ]:
train.info()

### Média estradas afetadas por gravidade de incidente

In [ ]:

sns.barplot(data=train, x="incidents", y="total_estradas")

### Hora com mais incidentes

In [ ]:
sns.histplot(data=train, x="Hour", hue="incidents", multiple="dodge")

### Mês com mais incidentes

In [ ]:
sns.histplot(data=train, x="Month", hue="incidents", multiple="dodge")

### Delay pelo número de estradas afetadas

In [ ]:
sns.lineplot(data=train, x="total_estradas", y="delay_in_seconds")

### Correlation Matrix

In [ ]:
corr_matrix = train.corr()
f , ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, vmin=-1, vmax=1, square=True, annot=True)

# Model


In [ ]:


X = train.drop(['incidents'], axis=1)
y = train['incidents']

X.to_csv('X.csv')


x_train, x_test, y_train, y_test = skl.model_selection.train_test_split(X,y, test_size=0.3, random_state=2022)   # using 30% for testing and 70% for training


### Logistic Classification

In [ ]:
# vamos entao dar fit nos dados de treino e prever com os de teste
lg = LogisticRegression(max_iter=100000,random_state=2022)
lg.fit(x_train, y_train)    #eventualmente podemos mudar o solver!
predictions = lg.predict(x_test)
score = lg.score(x_test, y_test)
print('\n\n\nScore with train data: ' + str(score) + '\n\n')
plot_confusion_matrix(lg, x_test, y_test)

### Random Forest

In [ ]:


#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=1000, random_state=2022)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred=clf.predict(x_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
plot_confusion_matrix(lg, x_test, y_test)


In [ ]:

from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=700,
                          learning_rate=1,
                          depth=4,
                          random_state=2022)
# Fit model
model.fit(x_train,y_train)

y_pred=model.predict(x_test)
# Get predictions
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
plot_confusion_matrix(lg, x_test, y_test)


In [ ]:
base_cls = DecisionTreeClassifier()

model = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = 1000,
                          random_state = 2022,
                          max_samples = 0.7)


model.fit(x_train,y_train)

y_pred = model.predict(x_test)

# Get predictions

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
plot_confusion_matrix(lg, x_test, y_test)

predictions = model.predict(test)




res = pd.DataFrame(data={'RowId': range(1,1207), 'Incidents': predictions})
res.to_csv('res.csv', index=False)
